## Reference:
[VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION](https://arxiv.org/pdf/1409.1556.pdf)


In [ ]:
# VGGNet 11/13/16/19 layer with pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.init as init

In [ ]:
# kernal size hasn't any change
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_layers, init_weights=True):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_layers])
        self.classifier = nn.Sequential(
            nn.Linear(512, 1200),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1200, 10),
        )
        if init_weights:
            self._initialize_weights()
    def forward(self, x):
        out = self.features(x)
        # flatten
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
    
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                layers += [conv2d, 
                           nn.BatchNorm2d(v), 
                           nn.ReLU(inplace=True)]
                in_channels = v
        # The asterisk interprets the information in the list
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight, mode = 'fan_out')
                if m.bias is not None:
                    init.constant(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant(m.weight, 1)
                init.constant(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal(m.weight, std=1e-2)
                if m.bias is not None:
                    init.constant(m.bias, 0)

# net = VGG('VGG19')
# x= torch.randn(64, 3, 32, 32)
# # params = list(net.parameters())
# # print(len(params))
# # print(params[0])
# print( net, net(Variable(x)).size())